In [30]:
#Importing all libraries needed.

%matplotlib inline


import datetime
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.metrics import roc_auc_score
from scipy.io import loadmat
from operator import itemgetter
import random
import os
import time
import glob
import re
from multiprocessing import Process
import copy

#Importing old and new Kfold
from sklearn.cross_validation import KFold
from sklearn.model_selection import KFold as NewKF

#Importing GroupKfold, only available since version 0.18
from sklearn.model_selection import GroupKFold


#Importing function for scaling data before PCA
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import scale

#Importing PCA packages
from sklearn.decomposition import PCA, KernelPCA

#Importing FFT package
from scipy.fftpack import fft

#Importing crossvalidation metrics and Gridsearch
from sklearn import cross_validation, metrics
from sklearn.model_selection import GridSearchCV

#Importing wrapper to use XGB with Gridsearch

from xgboost.sklearn import XGBClassifier

#Importing plotting packages (optional)

import matplotlib.pylab as plt

from pandas.tools.plotting import scatter_matrix

from sklearn.model_selection import LeavePGroupsOut
from sklearn.model_selection import GroupShuffleSplit


#Defining general modules used in the classification

random.seed(2016)
np.random.seed(2016)


def natural_key(string_):
    return [int(s) if s.isdigit() else s for s in re.split(r'(\d+)', string_)]


def create_feature_map(features):
    outfile = open('xgb.fmap', 'w')
    for i, feat in enumerate(features):
        outfile.write('{0}\t{1}\tq\n'.format(i, feat))
    outfile.close()


def get_importance(gbm, features):
    create_feature_map(features)
    importance = gbm.get_fscore(fmap='xgb.fmap')
    importance = sorted(importance.items(), key=itemgetter(1), reverse=True)
    return importance


def intersect(a, b):
    return list(set(a) & set(b))


def print_features_importance(imp):
    for i in range(len(imp)):
        print("# " + str(imp[i][1]))
        print('output.remove(\'' + imp[i][0] + '\')')


def mat_to_pandas(path):
    mat = loadmat(path)
    names = mat['dataStruct'].dtype.names
    ndata = {n: mat['dataStruct'][n][0, 0] for n in names}
    sequence = -1
    if 'sequence' in names:
        sequence = mat['dataStruct']['sequence']
    return pd.DataFrame(ndata['data'], columns=ndata['channelIndices'][0]), sequence
    


In [6]:
#Modules to read train and test data.


def create_simple_csv_train(patient_id):

    out = open("simple_train_" + str(patient_id) +"-short" + ".csv", "w")
    out.write("Id,sequence_id,patient_id")
    for i in range(16):
        out.write(",avg_" + str(i) + ",peak1_" + str(i) + ",peak2_" + str(i) + ",peak3_" + str(i) +",peak4_" + str(i))
    out.write(",file_size,result\n")

    # TRAIN (0)
    out_str = ''
    files = sorted(glob.glob("./data/train_" + str(patient_id) + "/*.mat"), key=natural_key)
    print ('train files'+ str(patient_id), len(files))    
    pos1=0
    neg1=0
    sequence_id = int(patient_id)*1000
    total = 0
    seq1=0
    for fl in files:
        total += 1
        # print('Go for ' + fl)
        id_str = os.path.basename(fl)[:-4]
        arr = id_str.split("_")
        patient = int(arr[0])
        id = int(arr[1])
        result = int(arr[2])
        new_id = patient*100000 + id
        try:
            tables, sequence_from_mat = mat_to_pandas(fl)
        except:
            print('Some error here {}...'.format(fl))
            continue
        out_str += str(new_id) + "," + str(sequence_id) + "," + str(patient)

        sizesignal=int(tables.shape[0])       
        
        for f in sorted(list(tables.columns.values)):
            mean = tables[f].mean()
            
            yf1 = fft(tables[f])
            fftpeak=2.0/sizesignal * np.abs(yf1[0:sizesignal/2])
 
            numberofbands=4

            sizeband=20/numberofbands

#           for i in range(numberofbands)
          
            peak1=fftpeak[0:5].mean()            
            peak2=fftpeak[5:10].mean()          
            peak3=fftpeak[10:15].mean()
            peak4=fftpeak[15:20].mean()
            
            out_str += "," + str(mean)+ "," + str(peak1) + "," + str(peak2) + "," + str(peak3) +"," + str(peak4)
        out_str += "," + str(os.path.getsize(fl)) + "," + str(result) + "\n"
        #print(sequence_from_mat)
        #print(type(sequence_from_mat))
        seq1=int(sequence_from_mat[0][0][0][0])
        print(total, seq1)
        if (total % 6 == 0) and (seq1==6):
            if result != 0:
                pos1 += 1
                print('Positive ocurrence sequence finished', pos1)
            else:
                neg1 += 1
                print('Negative ocurrence sequence finished', neg1)
                
            sequence_id += 1
            print ('sequence',sequence_id)

    out.write(out_str)
    
    out.close()
    print('Train CSV for patient {} has been completed...'.format(patient_id))


def create_simple_csv_test(patient_id):

    # TEST
    out_str = ''
    files = sorted(glob.glob("./data/test_" + str(patient_id) + "/*.mat"), key=natural_key)
    print ('test files'+ str(patient_id), len(files))    
    out = open("simple_test_" + str(patient_id) +"-short" + ".csv", "w")
    out.write("Id,patient_id")
    for i in range(16):
        out.write(",avg_" + str(i) + ",peak1_" + str(i) + ",peak2_" + str(i) + ",peak3_" + str(i) +",peak4_" + str(i))
    out.write(",file_size\n")
    for fl in files:
        # print('Go for ' + fl)
        id_str = os.path.basename(fl)[:-4]
        arr = id_str.split("_")
        patient = int(arr[0])
        id = int(arr[1])
        new_id = patient*100000 + id
        try:
            tables, sequence_from_mat = mat_to_pandas(fl)
        except:
            print('Some error here {}...'.format(fl))
            continue
        out_str += str(new_id) + "," + str(patient)

        sizesignal=int(tables.shape[0])           
        
        for f in sorted(list(tables.columns.values)):
            mean = tables[f].mean()
            
            yf1 = fft(tables[f])
            fftpeak=2.0/sizesignal * np.abs(yf1[0:sizesignal/2])

            numberofbands=4

            sizeband=20/numberofbands

#           for i in range(numberofbands)
          
            peak1=fftpeak[0:5].mean()            
            peak2=fftpeak[5:10].mean()          
            peak3=fftpeak[10:15].mean()
            peak4=fftpeak[15:20].mean()
            
            out_str += "," + str(mean)+ "," + str(peak1) + "," + str(peak2) + "," + str(peak3) +"," + str(peak4)
                        
        out_str += "," + str(os.path.getsize(fl)) + "\n"
        # break

    out.write(out_str)
    out.close()
    print('Test CSV for patient {} has been completed...'.format(patient_id))


In [31]:
#This is the main section. It is ordered as follows (it will change with each iteration, including comments format):
#
#      1. Variables and parameters for XGB training.
#      2. GridSearch.
#      3. XGB training (not linked to GRidsearch ouput yet)
#
#

def run_kfold(nfolds, train, test, features, target, random_state=2016):

    # Parameters and variables used later by xgb (not to be mixed with XGBClassifier used for GridSearch)
    
    num_boost_round = 1000
    early_stopping_rounds = 50
    
    train_index_group=[]
    test_index_group=[]
    
    yfull_train = dict()
    yfull_test = copy.deepcopy(test[['Id']].astype(object))
    
    num_fold = 0
    eta = 0.2
    max_depth = 5
    subsample = 0.7
    colsample_bytree = 0.8
    start_time = time.time()

    print('XGBoost params. ETA: {}, MAX_DEPTH: {}, SUBSAMPLE: {}, COLSAMPLE_BY_TREE: {}'.format(eta, max_depth, subsample, colsample_bytree))
    params = {
        "objective": "binary:logistic",
        "booster" : "gbtree",
        "eval_metric": "auc",
        "eta": eta,
        "tree_method": 'exact',
        "max_depth": max_depth,
        "subsample": subsample,
        "colsample_bytree": colsample_bytree,
        "silent": 1,
        "seed": random_state,
        "gamma": 0.3,
        "min_child_weight": 1
    }


    unique_sequences = np.array(train['sequence_id'].unique())
    print('unique sequences', unique_sequences)
    
    #This section is largely tentative. Contains variables that are not used rigth now (but could). Is left here
    #in case you find a need for them. I will use them in the next iterations of the code.
    
    groups1=np.fix(unique_sequences/1000)
    
    groups2=groups1.astype(int)
    print('groups', groups2)
    
    #Here there are a set of options for the split.
    
    gkf = GroupKFold(n_splits=3)
    test1=gkf.split(unique_sequences, groups=groups2)
    test2=gkf.split(unique_sequences, groups=groups2)
    
    kf = KFold(len(unique_sequences), n_folds=nfolds, shuffle=True, random_state=random_state)

    
    #Here starts the GRid search code. Most of it is not used for this script version. Again, still testing.
    #For grid search this script is using GroupShuffleSplit using the sequences as groups(iterator is assigned to 'custom_cv'
    #before performing the Grid Search)

    num_fold1=0
    param_test1 = {'max_depth': [3,5]}
#    param_test1 = {'max_depth': [3,5,7,9], 'min_child_weight':[1,3,5,7]}
#    param_test1 = {'max_depth': [3,5,7,9], 'gamma':[i/10.0 for i in range(0,5)]}
#    param_test1 = { 'subsample':[i/10.0 for i in range(6,10)],'colsample_bytree':[i/10.0 for i in range(6,10)]}  
#    param_test1 = { 'subsample':[i/10.0 for i in range(6,10)],'colsample_bytree':[i/10.0 for i in range(6,10)]}      

    for train_seq_index1, test_seq_index1 in kf:
        num_fold1 += 1
        print('this is creation of Kfold iterator')
        print('Start fold {} from {}'.format(num_fold1, nfolds))
    
        train_seq1 = unique_sequences[train_seq_index1]
        valid_seq1 = unique_sequences[test_seq_index1]

        train_index = train[train['sequence_id'].isin(train_seq1)].index.values
        test_index = train[train['sequence_id'].isin(valid_seq1)].index.values

        print(train_index, type(train_index))
        print(test_index, type(test_index))
        
        train_index_group.append(train_index)
        test_index_group.append(test_index)
    
    
    print('train index group',train_index_group)
    
#   custom_cv = [(train_index_group[i], test_index_group[i]) for i in range(0,6) ]

    number_groups=int(len(unique_sequences)/nfolds)
    print('groups',len(unique_sequences),number_groups)
    custom_cv=GroupShuffleSplit(n_splits=nfolds, test_size=0.2, random_state=0)
#   custom_cv=lpgo.split(train[features], train[target], groups=train['sequence_id'])

    print('custom cv', custom_cv)
                   

#    Scaling and PCA fro Grid search only


    scaler1 = MinMaxScaler() 
    
    train_features=train[features]
    train_target=train[target]
            
    train_scaled=pd.DataFrame(scaler1.fit_transform(train_features), columns=train_features.columns, index=train_features.index)


    pcatest=KernelPCA(n_components=30, kernel='poly')
    train_features_f=pd.DataFrame(pcatest.fit_transform(train_scaled), index=train_scaled.index)


    dmfeatures=train_features_f
    dmtarget=train_target
    
#   Grid search
    
    classifier1=XGBClassifier( learning_rate =0.2, n_estimators=1000, max_depth=5,
        min_child_weight=1, gamma=0.3, subsample=0.7, colsample_bytree=0.8,
        objective= 'binary:logistic', nthread=4, scale_pos_weight=1, seed=27)

    gsearch1 = GridSearchCV(estimator = classifier1, param_grid = param_test1, scoring='roc_auc',n_jobs=-1,iid=False, cv=custom_cv)
  
    
    gsearch1.fit(dmfeatures,dmtarget,groups=train['sequence_id'])

    print('best parameters, scores')    
    print(gsearch1.grid_scores_, gsearch1.best_params_, gsearch1.best_score_)


    
#   XGB training   

#   Here starts the XGB training. Is not being fed the parameters found above with Grid Search.
#   Some parameter optimization has been done manually. When you check the final score is rather
#   encouraging (72%) which has been attained by tweaking the hyper parameters with grid search manually
#   However, is being done on only part of the samples. Next is to apply this same code and manually 
#   change the same hyper parameters with the total data set. That takes more time.
    
    
    for train_seq_index, test_seq_index in kf:
        num_fold += 1
        print('Start fold {} from {}'.format(num_fold, nfolds))
        train_seq = unique_sequences[train_seq_index]
        valid_seq = unique_sequences[test_seq_index]
        print('Length of train people: {}'.format(len(train_seq)))
        print('Length of valid people: {}'.format(len(valid_seq)))

        X_train, X_valid = train[train['sequence_id'].isin(train_seq)][features], train[train['sequence_id'].isin(valid_seq)][features]
        y_train, y_valid = train[train['sequence_id'].isin(train_seq)][target], train[train['sequence_id'].isin(valid_seq)][target]
        X_test = test[features]

        print('Length train:', len(X_train))
        print('Length valid:', len(X_valid))
        
#       Scaling for PCA
        
        scaler = MinMaxScaler()   
        
        Xtrain_scaled=pd.DataFrame(scaler.fit_transform(X_train), columns=X_train.columns, index=X_train.index)
        Xvalid_scaled=pd.DataFrame(scaler.fit_transform(X_valid), columns=X_valid.columns, index=X_valid.index )

        Xtest_scaled=pd.DataFrame(scaler.fit_transform(X_test), columns=X_test.columns)
        

        
#       PCA transformation 
        pcatest=KernelPCA(n_components=30, kernel='poly')
        X_train_f=pd.DataFrame(pcatest.fit_transform(Xtrain_scaled), index=Xtrain_scaled.index)
        X_valid_f=pd.DataFrame(pcatest.fit_transform(Xvalid_scaled), index=Xvalid_scaled.index)

        X_test_f=pd.DataFrame(pcatest.fit_transform(Xtest_scaled))


        y_train_f=y_train
        y_valid_f=y_valid

        
#       Preparation for XGB training

        dtrain = xgb.DMatrix(X_train_f, y_train_f)
        dvalid = xgb.DMatrix(X_valid_f, y_valid_f)

        watchlist = [(dtrain, 'train'), (dvalid, 'eval')]       
        
        gbm = xgb.train(params, dtrain, num_boost_round, evals=watchlist, early_stopping_rounds=early_stopping_rounds, verbose_eval=500)

        yhat = gbm.predict(xgb.DMatrix(X_valid_f), ntree_limit=gbm.best_iteration+1)

#       Each time store portion of precicted data in train predicted values

        for i in range(len(X_valid_f.index)):
            yfull_train[X_valid_f.index[i]] = yhat[i]
            
        print("Validating...")
        check = gbm.predict(xgb.DMatrix(X_valid_f), ntree_limit=gbm.best_iteration+1)
        score = roc_auc_score(y_valid.tolist(), check)
        print('Check error value: {:.6f}'.format(score))

        print("Predict test set...")
        test_prediction1 = gbm.predict(xgb.DMatrix(X_test_f), ntree_limit=gbm.best_iteration+1)
        yfull_test['kfold_' + str(num_fold)] = test_prediction1
        
              

    # Copy dict to list
    train_res = []
    
    for i in range(len(train.index)):
        train_res.append(yfull_train[i])

    score = roc_auc_score(train[target], np.array(train_res))
    print('Check error value: {:.6f}'.format(score))

    # Find mean for KFolds on test
    merge = []
    for i in range(1, nfolds+1):
        merge.append('kfold_' + str(i))
    yfull_test['mean'] = yfull_test[merge].mean(axis=1)

    print('Training time: {} minutes'.format(round((time.time() - start_time)/60, 2)))
    return yfull_test['mean'].values, score



In [3]:
# These modules are self-explanatory. They are used by other modules in reading data and creating final submission. 
# they are largely fixed.

def create_submission(score, test, prediction):
    # Make Submission
    now = datetime.datetime.now()
    sub_file = 'submission_' + str(score) + '_' + str(now.strftime("%Y-%m-%d-%H-%M")) + '.csv'
    print('Writing submission: ', sub_file)
    f = open(sub_file, 'w')
    f.write('File,Class\n')
    total = 0
    for id in test['Id']:
        patient = id // 100000
        fid = id % 100000
        str1 = str(patient) + '_' + str(fid) + '.mat' + ',' + str(prediction[total])
        str1 += '\n'
        total += 1
        f.write(str1)
    f.close()


def get_features(train, test):
    trainval = list(train.columns.values)
    testval = list(test.columns.values)
    output = intersect(trainval, testval)
    output.remove('Id')
    # output.remove('file_size')
    return sorted(output)


def read_test_train():
    print("Load train.csv...")
    train1 = pd.read_csv("simple_train_1-short.csv")
    train2 = pd.read_csv("simple_train_2-short.csv")
    train3 = pd.read_csv("simple_train_3-short.csv")
    train = pd.concat([train1, train2, train3])
    # Remove all zeroes files
    train = train[train['file_size'] > 55000].copy()
    # Shuffle rows since they are ordered
    train = train.iloc[np.random.permutation(len(train))]
    # Reset broken index
    train = train.reset_index()
    print("Load test.csv...")
    test1 = pd.read_csv("simple_test_1-short.csv")
    test2 = pd.read_csv("simple_test_2-short.csv")
    test3 = pd.read_csv("simple_test_3-short.csv")
    test = pd.concat([test1, test2, test3])
    print("Process tables...")
    features = get_features(train, test)
    return train, test, features
    

 

In [ ]:
#  This section creates the train and test files, then reads them and concatenates them
#  in two large train and test file.

if __name__ == '__main__':
    print('XGBoost: {}'.format(xgb.__version__))
    if 1:
        # Do reading and processing of MAT files in parallel
        p = dict()
        p[1] = Process(target=create_simple_csv_train, args=(1,))
        p[1].start()
        p[2] = Process(target=create_simple_csv_train, args=(2,))
        p[2].start()
        p[3] = Process(target=create_simple_csv_train, args=(3,))
        p[3].start()
        p[4] = Process(target=create_simple_csv_test, args=(1,))
        p[4].start()
        p[5] = Process(target=create_simple_csv_test, args=(2,))
        p[5].start()
        p[6] = Process(target=create_simple_csv_test, args=(3,))
        p[6].start()
        p[1].join()
        p[2].join()
        p[3].join()
        p[4].join()
        p[5].join()
        p[6].join()
    train, test, features = read_test_train()
    print('Length of train: ', len(train))
    print('Length of test: ', len(test))
    print('Features [{}]: {}'.format(len(features), sorted(features)))
  

In [8]:
# This section only loads and concatenates the data from existing files. Unless you need to add 'features'
# or modify columns, this is the section to run to read the data.

if __name__ == '__main__':
    print('XGBoost: {}'.format(xgb.__version__))
    
    train, test, features = read_test_train()
    print('Length of train: ', len(train))
    print('Length of test: ', len(test))
    print('Features [{}]: {}'.format(len(features), sorted(features)))
       

XGBoost: 0.6
Load train.csv...
Load test.csv...
Process tables...
Length of train:  486
Length of test:  303
Features [82]: ['avg_0', 'avg_1', 'avg_10', 'avg_11', 'avg_12', 'avg_13', 'avg_14', 'avg_15', 'avg_2', 'avg_3', 'avg_4', 'avg_5', 'avg_6', 'avg_7', 'avg_8', 'avg_9', 'file_size', 'patient_id', 'peak1_0', 'peak1_1', 'peak1_10', 'peak1_11', 'peak1_12', 'peak1_13', 'peak1_14', 'peak1_15', 'peak1_2', 'peak1_3', 'peak1_4', 'peak1_5', 'peak1_6', 'peak1_7', 'peak1_8', 'peak1_9', 'peak2_0', 'peak2_1', 'peak2_10', 'peak2_11', 'peak2_12', 'peak2_13', 'peak2_14', 'peak2_15', 'peak2_2', 'peak2_3', 'peak2_4', 'peak2_5', 'peak2_6', 'peak2_7', 'peak2_8', 'peak2_9', 'peak3_0', 'peak3_1', 'peak3_10', 'peak3_11', 'peak3_12', 'peak3_13', 'peak3_14', 'peak3_15', 'peak3_2', 'peak3_3', 'peak3_4', 'peak3_5', 'peak3_6', 'peak3_7', 'peak3_8', 'peak3_9', 'peak4_0', 'peak4_1', 'peak4_10', 'peak4_11', 'peak4_12', 'peak4_13', 'peak4_14', 'peak4_15', 'peak4_2', 'peak4_3', 'peak4_4', 'peak4_5', 'peak4_6', 'pe

In [32]:
#Here is where the actual prediction and submission are executed and created respectively.

prediction, score = run_kfold(6, train, test, features, 'result')

create_submission(score, test, prediction)
print('version 3acb')

unique sequences [3014 3006 2005 1006 2007 3012 2004 2000 1003 2001 2014 1013 3000 1008 3007
 1004 2006 3008 3009 1017 3013 1014 3005 2015 1002 2012 1009 1001 2003 2008
 1000 3001 2002 2009 1005 2010 2013 3015 1012 3004 2011 3003 3017 3002 3010
 1010 2016 1007 2017 1011 1015 3016 3011 1016]
groups [3 3 2 1 2 3 2 2 1 2 2 1 3 1 3 1 2 3 3 1 3 1 3 2 1 2 1 1 2 2 1 3 2 2 1 2 2
 3 1 3 2 3 3 3 3 1 2 1 2 1 1 3 3 1]
XGBoost params. ETA: 0.2, MAX_DEPTH: 5, SUBSAMPLE: 0.7, COLSAMPLE_BY_TREE: 0.8
this is creation of Kfold iterator
Start fold 1 from 6
[  0   2   4   5   8   9  10  11  12  13  17  18  19  21  22  24  25  26
  28  29  30  31  32  33  35  36  37  38  39  40  41  42  43  45  46  47
  48  49  50  51  52  53  55  56  57  59  60  61  62  63  64  66  67  70
  71  72  73  74  75  76  77  78  80  81  82  84  85  86  87  89  90  91
  92  93  94  96  97  99 100 101 102 103 105 106 108 111 112 114 115 116
 117 118 120 122 123 124 125 127 129 131 132 133 134 135 136 138 140 142
 143 146 147 148 1

/home/rodolfoxps/anaconda3/lib/python3.5/site-packages/sklearn/model_selection/_search.py:662: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


Stopping. Best iteration:
[6]	train-auc:0.999941	eval-auc:0.889583

Validating...
Check error value: 0.889583
Predict test set...
Start fold 2 from 6
Length of train people: 45
Length of valid people: 9
Length train: 413
Length valid: 73
[0]	train-auc:0.957497	eval-auc:0.605751
Multiple eval metrics have been passed: 'eval-auc' will be used for early stopping.

Will train until eval-auc hasn't improved in 50 rounds.
Stopping. Best iteration:
[3]	train-auc:0.994793	eval-auc:0.642788

Validating...
Check error value: 0.642788
Predict test set...
Start fold 3 from 6
Length of train people: 45
Length of valid people: 9
Length train: 403
Length valid: 83
[0]	train-auc:0.962344	eval-auc:0.408805
Multiple eval metrics have been passed: 'eval-auc' will be used for early stopping.

Will train until eval-auc hasn't improved in 50 rounds.
Stopping. Best iteration:
[6]	train-auc:0.998072	eval-auc:0.598742

Validating...
Check error value: 0.598742
Predict test set...
Start fold 4 from 6
Length of 